In [33]:
import numpy as np
from numpy import pi
import kwant
import kwant.continuum
import tinyarray
import scipy.sparse.linalg as sla
from LK_utils import consts, Jmatrices, H_LKBP, box_3D, make_model_
from material_parameters import parameters
from double_dot import 

# define box of computation (lengths in nm)

L = 120
W = 120
H = 20

# Parameter for definition of computation lattice

ax, ay, az = 4, 4, 1

lat_params = {"L": L, "W": W, "H": H, "ax": ax, "ay": ay, "az": az}
print('Lattice parameters (lengths in nm): {}'.format(lat_params))

nx, ny, nz = int(L/ax - 1), int(W/ay - 1), int(H/az - 1)
print('Number of points on the lattice: {0}*{1}*{2}={3}'.format(nx, ny, nz, nx*ny*nz))



Lattice parameters (lengths in nm): {'L': 120, 'W': 120, 'H': 20, 'ax': 4, 'ay': 4, 'az': 1}
Number of points on the lattice: 29*29*19=15979

    mu * ((ga1 + 5*ga2/2) * (k_x**2 + k_y**2 + k_z**2 + 2 * k_x * Ax(y,z) + 2 * k_y * Ay(x,z)) * id4 
    - 2 * ga2 * (k_x**2 * Jx2 + k_y**2 * Jy2 + k_z**2 * Jz2 + 2 * k_x * Ax(y,z) * Jx2 + 2 * k_y * Ay(x,z) * Jy2) 
    - ga3 * (kyz * Jyz + kzx * Jzx + kxy * Jxy 
    + (k_z * Ay(x,z) + Ay(x,z) * k_z) * Jyz + (k_z * Ax(y,z) + Ax(y,z) * k_z) * Jzx 
    + (k_x * Ay(x,z) + Ax(y,z) * k_y + k_y * Ax(y,z) + Ay(x,z) * k_x) * Jxy)) 
    + 2 * muB * (ka * (Jx*Bx + Jy*By + Jz*Bz) + qu * (Jx3*Bx + Jy3*By + Jz3*Bz)) 
    + V(x, y, z) * id4
    + bv * (epsxx * Jx2 + epsyy * Jy2 + epszz * Jz2)
    


In [29]:
def box_3D(site):
    (x, y, z) = site.pos
    return 0 <= x < L - ax and 0 <= y < W - ay and 0 <= z < H - az


def make_model_LKBP(H_LK, mat_params, lat_params, eps_para, eps_perp, B):
    syst = kwant.Builder()
    subs = consts
    subs.update(mat_params)
    subs.update(lat_params)
    subs.update(Jmatrices)
    subs.update({
        "Bx": B[0], "By": B[1], "Bz": B[2],
        "kyz": "k_y * k_z + k_z * k_y",
        "kzx": "k_z * k_x + k_x * k_z",
        "kxy": "k_x * k_y + k_y * k_x",
        "epsxx": eps_para, "epsyy": eps_para, "epszz":eps_perp
        })

    model = kwant.continuum.discretize(H_LK, locals=subs, grid=kwant.lattice.general([(ax, 0, 0), (0, ay, 0), (0, 0, az)], norbs=4))
    syst.fill(model, box_3D, (0, 0, 0))
    syst = syst.finalized()
    return syst


def eigs_gate_defined_potential(syst, d_QW, B, V, N=8):
    def potential_vector_x(y, z):
        return (2*pi/consts['phi0']) * (z*B[1] - y*B[2]/2)
    
    def potential_vector_y(x, z):
        return (2*pi/consts['phi0']) * (-z*B[0] + x*B[2]/2)
    
    def potential(x, y, z):
        def g(x,y,z):
            return (0.5/pi) * np.arctan2(x * y, (z * np.sqrt(x**2 + y**2 + z**2)))
        
        Left = x0 - 0.5*Lg # Left and right x-coordinates of the dot
        Right = x0 + 0.5*Lg  
        Bot = y0 - 0.5*Lg # Bottom and top y-coordinates of the dot
        Top = y0 + 0.5*Lg
        return V * (g(x - Left, y - Bot, z + d_QW) + g(x - Left, Top - y, z + d_QW) + g(Right - x, y - Bot, z + d_QW) + g(Right - x, Top - y, z + d_QW))
        
    Ham = syst.hamiltonian_submatrix(params=dict(V=potential, Ax=potential_vector_x, Ay=potential_vector_y), sparse=True)
    evals, evecs = sla.eigsh(Ham, k=N, which='SA')
    idx = evals.argsort()
    return [evals[idx], evecs[:,idx]]


def eigs_double_dot_potential(syst, B, d, s, U0, eEz, N=8):
    def potential_vector_x(y, z):
        return (2*pi/consts['phi0']) * (z*B[1] - y*B[2])
    
    def potential_vector_y(x, z):
        return (2*pi/consts['phi0']) * (-z*B[0])
    
    def double_dot_potential(x, y, z):
        return Gaussian_symmetric_double_dot(x, y, d, U0, s) + eEz*z

    Ham = syst.hamiltonian_submatrix(params=dict(V=double_dot_potential, Ax=potential_vector_x, Ay=potential_vector_y), sparse=True)
    evals, evecs = sla.eigsh(Ham, k=N, which='SA')
    idx = evals.argsort()
    return [evals[idx], evecs[:,idx]]


In [4]:
mat = 'Ge'

def get_material_parameters(mat):
    for row in parameters:
        if row["Material"] == mat:
            return row

mat_params = get_material_parameters(mat)
print('Material parameters: {}'.format(mat_params))


Material parameters: {'Material': 'Ge', 'ga1': 13.38, 'ga2': 4.24, 'ga3': 5.69, 'ka': 3.41, 'qu': 0.06, 'bv': -2860.0, 'nu': 0.73}


In [5]:
import time

# gate position and dimension in nm

x0 = L/2
y0 = W/2
Lg = 80

# Distance to quantum well in nm

d_QW = 50

# Strain parameters

eps_para = -0.0061
eps_perp = 0.0045

def Delta_s(mat_params, lat_params, eps_para, eps_perp, B, V):
    print('Computing spin splitting for magnetic field B = {0} T and gate voltage Vg = {1:.1f} mV'.format(B, V))
    t1 = time.time()
    syst = make_model_LKBP(H_LKBP, mat_params, lat_params, eps_para, eps_perp, B)
    evals = eigs_gate_defined_potential(syst, d_QW, B, V)[0]
    Ds = evals[1] - evals[0]
    t2 = time.time()
    print('Spin splitting: {0:.6f} meV (done in {1:.2f}s).'.format(Ds, t2 - t1))
    return Ds

Delta_s(mat_params=mat_params, lat_params=lat_params, eps_para=eps_para, eps_perp=eps_perp, B=[0.25, 0.25, 0.05], V=-1000)

Computing spin splitting for magnetic field B = [0.25, 0.25, 0.05] T and gate voltage Vg = -1000.0 mV
Spin splitting: 0.039310 meV (done in 57.63s).


0.039310170102339725

In [6]:
def Ds_d01(mat_params, lat_params, B, V):
    """
    Computes spin splitting and electric dipole matrix matrix elements 
    """
    t1 = time.time()
    print('Computing spin splitting and electric dipole moments for magnetic field B = {0} T and gate voltage Vg = {1:.1f} mV'.format(B, V))
    syst = make_model_LKBP(H_LKBP, mat_params, lat_params, eps_para, eps_perp, B)
    eigs = eigs_gate_defined_potential(syst, d_QW, B, V)
    evals = eigs[0]
    Ds = evals[1] - evals[0]
    print('Ds = {0} meV'.format(Ds))
    
    norbs = 4
    x = np.arange(0, L - ax, ax)
    y = np.arange(0, W - ay, ay)
    z = np.arange(0, H - az, az)
    X, Y, Z = np.meshgrid(x, y, z, indexing='ij')
    
    grid_shape = np.shape(X)
    
    # Get the wavefunctions for the first two levels
    psi0 = eigs[1][:, 0].reshape((int(L/ax - 1) * int(W/ay - 1) * int(H/az - 1), norbs))
    psi1 = eigs[1][:, 1].reshape((int(L/ax - 1) * int(W/ay - 1) * int(H/az - 1), norbs))
    
    # Compute matrix element for each spinor component
    integral_x01 = integral_y01 = integral_z01 = integral_xl = integral_yl = integral_zl = 0.0 + 0.0j 
    norm0 = norm1 = 0.0
    
    for i in range(4):
        psi0_i = psi0[:, i].reshape(grid_shape)
        psi1_i = psi1[:, i].reshape(grid_shape)
        norm0 += np.sum(np.conj(psi0_i)*psi0_i)
        norm1 += np.sum(np.conj(psi1_i)*psi1_i)
        integral_x01 += np.sum(np.conj(psi0_i) * X * psi1_i)
        integral_y01 += np.sum(np.conj(psi0_i) * Y * psi1_i) 
        integral_z01 += np.sum(np.conj(psi0_i) * Z * psi1_i)
        integral_xl += np.sum(np.conj(psi1_i) * X * psi1_i - np.conj(psi0_i) * X * psi0_i) 
        integral_yl += np.sum(np.conj(psi1_i) * Y * psi1_i - np.conj(psi0_i) * Y * psi0_i) 
        integral_zl += np.sum(np.conj(psi1_i) * Z * psi1_i - np.conj(psi0_i) * Z * psi0_i)

    print(norm0, norm1)
    x01 = np.abs(integral_x01)
    y01 = np.abs(integral_y01)
    z01 = np.abs(integral_z01)
    xl = np.abs(integral_xl)
    yl = np.abs(integral_yl)
    zl = np.abs(integral_zl)
    t2 = time.time()
    print('x01 = {0:.5f} nm'.format(x01))
    print('y01 = {0:.5f} nm'.format(y01))
    print('z01 = {0:.5f} nm'.format(z01))
    print('xl = {0:.5f} nm'.format(xl))
    print('yl = {0:.5f} nm'.format(yl))
    print('zl = {0:.5f} nm'.format(zl))
    print('Done in {0:3f} s'.format(t2-t1))
    return [Ds, x01, y01, z01, xl, yl, zl]

Ds_d01(mat_params=mat_params, lat_params=lat_params, B=[0.25, 0.25, 0.05], V=-1000)

Computing spin splitting and electric dipole moments for magnetic field B = [0.25, 0.25, 0.05] T and gate voltage Vg = -1000.0 mV
Ds = 0.03931017010503979 meV
(1.0000000000001066-6.00033552236637e-19j) (1.0000000000001132-1.0627459017322388e-18j)
x01 = 0.00020 nm
y01 = 0.00020 nm
z01 = 0.00012 nm
xl = 0.00104 nm
yl = 0.00104 nm
zl = 0.00062 nm
Done in 62.658700 s


[0.03931017010503979,
 0.00019875208267864405,
 0.0001987520318934856,
 0.00012360939076862552,
 0.0010437415291164598,
 0.0010437415289104024,
 0.0006209230228169105]